<a href="https://colab.research.google.com/github/SilinMeng0510/DeepLearning/blob/main/MNIST/TensorFlow_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

In [8]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)


def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)


BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)


BATCH_SIZE = 64

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

In [13]:
input_size = 784
output_size = 10
hidden_layer_size = 400

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),
                            tf.keras.layers.Dense(600, activation='relu', kernel_initializer=tf.keras.initializers.HeNormal()),
                            tf.keras.layers.Dense(200, activation='relu', kernel_initializer=tf.keras.initializers.HeNormal()),
                            tf.keras.layers.Dense(output_size, activation='softmax', kernel_initializer=tf.keras.initializers.GlorotUniform())   
                            ])

In [14]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [15]:
NUM_EPOCHS = 7

model.fit(train_data, epochs = NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), 
          validation_steps=10, verbose=2)

Epoch 1/7
844/844 - 12s - loss: 0.2051 - accuracy: 0.9377 - val_loss: 0.0909 - val_accuracy: 0.9733 - 12s/epoch - 14ms/step
Epoch 2/7
844/844 - 13s - loss: 0.0787 - accuracy: 0.9753 - val_loss: 0.0652 - val_accuracy: 0.9812 - 13s/epoch - 15ms/step
Epoch 3/7
844/844 - 12s - loss: 0.0529 - accuracy: 0.9826 - val_loss: 0.0526 - val_accuracy: 0.9842 - 12s/epoch - 14ms/step
Epoch 4/7
844/844 - 12s - loss: 0.0393 - accuracy: 0.9871 - val_loss: 0.0413 - val_accuracy: 0.9880 - 12s/epoch - 14ms/step
Epoch 5/7
844/844 - 11s - loss: 0.0296 - accuracy: 0.9902 - val_loss: 0.0480 - val_accuracy: 0.9872 - 11s/epoch - 13ms/step
Epoch 6/7
844/844 - 12s - loss: 0.0271 - accuracy: 0.9916 - val_loss: 0.0350 - val_accuracy: 0.9878 - 12s/epoch - 14ms/step
Epoch 7/7
844/844 - 11s - loss: 0.0223 - accuracy: 0.9927 - val_loss: 0.0248 - val_accuracy: 0.9928 - 11s/epoch - 13ms/step


In [16]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 1s 1s/step - loss: 0.0679 - accuracy: 0.9840


In [7]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.09. Test accuracy: 97.93%
